<a href="https://colab.research.google.com/github/ml-utils/POLAR/blob/master/Copy_of_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!ls -lh
!wget -nc https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip -n glove.42B.300d.zip
!ls -lh

total 4.0K
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data
--2022-02-19 17:46:55--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-02-19 17:46:55--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  5.20MB/s    in 5m 54s  

2022-02-19 17:52:49 (5.06 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

Archive:  glove

In [30]:
!pip install ipython-autotime
!wget -nc https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/LenciBenotto.val -P ./Antonym_sets
!wget -nc https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/LenciBenotto.test -P ./Antonym_sets
!wget -nc https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/EVALution.val -P ./Antonym_sets
!wget -nc https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/EVALution.test -P ./Antonym_sets
!ls -lh ./Antonym_sets/

--2022-02-19 17:53:48--  https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/LenciBenotto.val
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16922 (17K) [text/plain]
Saving to: ‘./Antonym_sets/LenciBenotto.val’

LenciBenotto.val    100%[===================>]  16.53K  --.-KB/s    in 0s      

2022-02-19 17:53:48 (109 MB/s) - ‘./Antonym_sets/LenciBenotto.val’ saved [16922/16922]

--2022-02-19 17:53:48--  https://raw.githubusercontent.com/Sandipan99/POLAR/master/Antonym_sets/LenciBenotto.test
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting 

In [31]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath
%load_ext autotime

time: 105 µs (started: 2022-02-19 17:53:49 +00:00)


In [32]:
import pickle
import os
import gc

orthogonal_antonymy_vector_filename = 'orthogonal_antonymy_vector.pkl'
dimension_similarity_matrix_filename = 'dimension_similarity_matrix.pkl'
model_gn_filename = 'model_gn.pkl'
embedding_matrix_np_filename = 'embedding_matrix_np'
model_gn_filename = 'model_gn.pkl'
embedding_matrix_np_dir = './embedding_matrix_np/'

def npsave(obj, filename):
    with open(filename, 'wb') as f:
      np.save(f, obj)  

def npload(filename):
    with open(filename, 'rb') as f:
      obj = np.load(f, allow_pickle=True)
    return obj

!ls -lh

total 6.5G
drwxr-xr-x 2 root root 4.0K Feb 19 17:53 Antonym_sets
-rw-rw-r-- 1 root root 4.7G Oct 24  2015 glove.42B.300d.txt
-rw-r--r-- 1 root root 1.8G Oct 25  2015 glove.42B.300d.zip
drwxr-xr-x 1 root root 4.0K Feb  1 14:32 sample_data
time: 127 ms (started: 2022-02-19 17:53:49 +00:00)


# load embeddings and normalizing them

In [33]:
model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt') ## needed for glove embeddings only

time: 30.1 s (started: 2022-02-19 17:53:49 +00:00)


In [34]:
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False) ## needed for only glove

time: 5min 37s (started: 2022-02-19 17:54:20 +00:00)


In [35]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

time: 6.52 ms (started: 2022-02-19 17:59:57 +00:00)


In [36]:
generate_norm_embedding(model_glove,'glove_norm_300.mod')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 34.8 s (started: 2022-02-19 17:59:57 +00:00)


### Load the required word-embeddings

In [37]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss

time: 27.4 s (started: 2022-02-19 18:00:32 +00:00)


In [38]:
current_model = model_gn 

with open(model_gn_filename, 'wb') as f:
    pickle.dump(model_gn, f)

time: 15.5 s (started: 2022-02-19 18:01:00 +00:00)


# Loading the antonyms

In [39]:
list_antonym = []
skipped_antonyms = []

def parse_antonym_set(filename):
  with open(filename) as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((bare(word1), bare(word2)))
            else:
              skipped_antonyms.append((bare(word1), bare(word2)))

def bare(word):
  return word.strip().lower()

parse_antonym_set('Antonym_sets/LenciBenotto.val')
parse_antonym_set('Antonym_sets/LenciBenotto.test')
parse_antonym_set('Antonym_sets/EVALution.val')
parse_antonym_set('Antonym_sets/EVALution.test')
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

# todo: print antonyms
# print skypped antonyms because not present in the model
print(f'list_antonym: {list_antonym}')
print(f'skipped_antonyms ({len(skipped_antonyms)}): {skipped_antonyms}')

list_antonym: [('implicate', 'exclude'), ('assertion', 'rejection'), ('terminal', 'beginning'), ('lop', 'secure'), ('pastor', 'libertine'), ('pocket', 'hook'), ('orbital', 'stationary'), ('evidence', 'disprove'), ('psychological', 'biological'), ('preparedness', 'unprepared'), ('saturation', 'dehydration'), ('geologic', 'biologic'), ('accessibility', 'secured'), ('fight', 'agree'), ('misconstrue', 'beautify'), ('lump', 'smooth'), ('double', 'half'), ('training', 'forgetting'), ('determined', 'negligent'), ('velocity', 'still'), ('shield', 'vulnerable'), ('measurement', 'guess'), ('iconic', 'ordinary'), ('thump', 'caress'), ('stretcher', 'compressor'), ('allude', 'distract'), ('cemetery', 'birthplace'), ('corrupt', 'sanctify'), ('cameraman', 'actor'), ('knowledgeable', 'uninformed'), ('screech', 'talk'), ('hedge', 'squander'), ('aggregate', 'please'), ('constrict', 'open'), ('make', 'break'), ('normative', 'descriptive'), ('inscribe', 'polish'), ('bulge', 'hollow'), ('edit', 'expand'), 

In [40]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)

print(f'similarity_matrix: {similarity_matrix}')

# calculates similarity among every two antonyms dimensions
all_similarity = defaultdict(dict)
antonyms_by_similarity_tmp = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
        # cosine_similarity([current_model[each_key]]
        word1_vec = current_model[each_key]
        word2_vec = current_model[each_value]
        cosine_sim = cosine_similarity([word1_vec],[word2_vec])
        # print(f'cosine similarity btw {each_key} and {each_value}: {cosine_sim}')
        all_similarity[each_key][each_value] = abs(cosine_sim[0][0])
        antonyms_by_similarity_tmp[all_similarity[each_key][each_value]] = (each_key, each_value)

# list the most dissimilar antonyms.. (is that a meaningful comparison?)
import collections
antonyms_by_similarity = collections.OrderedDict(reversed(sorted(antonyms_by_similarity_tmp.items())))


final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list
print(f'list_antonym: {list_antonym}')
print(f'antonyms_by_similarity: {antonyms_by_similarity}')
antonyms_by_similarity

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3110 [00:00<?, ?it/s]

similarity_matrix: defaultdict(<class 'list'>, {'exclude': ['implicate', 'inscribe'], 'assertion': ['rejection', 'meek', 'query', 'denial', 'lazy', 'question'], 'beginning': ['terminal'], 'lop': ['secure', 'trim'], 'libertine': ['pastor'], 'hook': ['pocket'], 'orbital': ['stationary', 'stable', 'suborbital'], 'disprove': ['evidence', 'document'], 'biological': ['psychological'], 'preparedness': ['unprepared', 'unreadiness', 'unpreparedness', 'unplanned'], 'dehydration': ['saturation'], 'biologic': ['geologic'], 'accessibility': ['secured', 'restricted', 'remoteness', 'hindrance', 'restriction', 'inaccessibility', 'unavailability'], 'agree': ['fight', 'battle', 'crab', 'contrast'], 'beautify': ['misconstrue'], 'lump': ['smooth', 'spread', 'separate'], 'double': ['half', 'single', 'indivisible'], 'forgetting': ['training'], 'determined': ['negligent', 'irresolute', 'undetermined', 'submissive', 'uncaring', 'lazy', 'undecided'], 'still': ['velocity'], 'shield': ['vulnerable', 'unshield', 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/1468 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1468 [00:00<?, ?it/s]

1468
list_antonym: [('exclude', 'inscribe'), ('assertion', 'meek'), ('beginning', 'terminal'), ('lop', 'secure'), ('libertine', 'pastor'), ('hook', 'pocket'), ('orbital', 'stable'), ('disprove', 'document'), ('biological', 'psychological'), ('preparedness', 'unreadiness'), ('dehydration', 'saturation'), ('biologic', 'geologic'), ('accessibility', 'hindrance'), ('agree', 'crab'), ('beautify', 'misconstrue'), ('lump', 'separate'), ('double', 'indivisible'), ('forgetting', 'training'), ('determined', 'irresolute'), ('still', 'velocity'), ('shield', 'unshield'), ('guess', 'tabulate'), ('iconic', 'undefined'), ('caress', 'punch'), ('compressor', 'stretcher'), ('allude', 'specify'), ('birthplace', 'cemetery'), ('corrupt', 'sanctify'), ('actor', 'cameraman'), ('knowledgeable', 'naivety'), ('screech', 'talk'), ('hedge', 'squander'), ('aggregate', 'scatter'), ('constrict', 'free'), ('break', 'construct'), ('descriptive', 'normative'), ('inscribe', 'smooth'), ('bulge', 'depress'), ('edit', 'unce

OrderedDict([(0.93784505, ('female', 'male')),
             (0.92661804, ('horizontal', 'vertical')),
             (0.8924475, ('north', 'south')),
             (0.8894876, ('east', 'west')),
             (0.88710755, ('they', 'we')),
             (0.87546605, ('nothing', 'something')),
             (0.86959004, ('he', 'she')),
             (0.8682928, ('down', 'up')),
             (0.86630225, ('gold', 'silver')),
             (0.86153054, ('large', 'small')),
             (0.8488299, ('blue', 'red')),
             (0.8461711, ('black', 'white')),
             (0.83778626, ('negative', 'positive')),
             (0.8320416, ('export', 'import')),
             (0.8315834, ('boyfriend', 'girlfriend')),
             (0.8295793, ('answer', 'question')),
             (0.82936513, ('everybody', 'nobody')),
             (0.8255644, ('boy', 'girl')),
             (0.82476544, ('give', 'take')),
             (0.8209698, ('long', 'short')),
             (0.8186827, ('favorable', 'unfavorable'))

time: 1.58 s (started: 2022-02-19 18:01:15 +00:00)


## Decide on the size of the antonym vector

In [41]:
# todo: compare two vectors, see which dimensions are both high (on the + or - sides), or both close to 0
# dims range should be +-2
def compare_2vectors(word1, word2):
  print(f'abs_diff {word1}, {word2}')
  word1_vec = current_model[each_key]
  word2_vec = current_model[each_value]
  sliced = np.vstack((word1_vec, word2_vec))

  dims_by_diff = defaultdict(dict)
  for dim in range(0, len(word1_vec)):
    dim_diff = abs(word2_vec[dim] - word1_vec[dim])
    dims_by_diff[dim_diff] = (word1_vec[dim], word2_vec[dim])

  dims_by_diff_sorted = collections.OrderedDict(reversed(sorted(dims_by_diff.items())))
  # print(f'{word1_vec[dim]} {word2_vec[dim]}')
  for idx, (diff, values) in enumerate(dims_by_diff_sorted.items()):
    print(f'{idx}   {diff:.4f} ({values[0]:.4f}, {values[1]:.4f})')
  print(dims_by_diff_sorted)
compare_2vectors('horizontal', 'vertical')

abs_diff horizontal, vertical
0   0.1836 (0.1629, -0.0207)
1   0.1768 (-0.1033, 0.0735)
2   0.1625 (-0.0579, 0.1046)
3   0.1599 (-0.1205, 0.0394)
4   0.1581 (0.0361, -0.1221)
5   0.1546 (-0.0931, 0.0615)
6   0.1516 (0.1057, -0.0460)
7   0.1428 (-0.0909, 0.0518)
8   0.1355 (-0.1131, 0.0224)
9   0.1337 (0.0936, -0.0401)
10   0.1331 (-0.0487, 0.0844)
11   0.1299 (-0.0820, 0.0479)
12   0.1296 (-0.0526, 0.0770)
13   0.1291 (-0.0989, 0.0302)
14   0.1281 (0.0979, -0.0302)
15   0.1269 (0.0185, -0.1085)
16   0.1266 (0.1569, 0.0303)
17   0.1240 (0.0468, -0.0772)
18   0.1240 (0.0912, -0.0328)
19   0.1234 (0.0118, -0.1115)
20   0.1189 (-0.1248, -0.0059)
21   0.1171 (-0.0180, 0.0990)
22   0.1139 (-0.0071, 0.1067)
23   0.1123 (-0.0002, 0.1121)
24   0.1123 (0.0128, 0.1250)
25   0.1123 (0.0820, -0.0303)
26   0.1122 (0.0330, -0.0793)
27   0.1113 (0.0414, -0.0699)
28   0.1108 (0.0444, -0.0663)
29   0.1102 (0.0726, -0.0376)
30   0.1100 (0.0578, -0.0522)
31   0.1094 (-0.0370, 0.0724)
32   0.1081 (-0.0689,

In [42]:
# num_antonym = 1468

time: 930 µs (started: 2022-02-19 18:01:17 +00:00)


In [68]:
# todo: another way to find interpretable dimension, instead of diff btw just a pair of antonyms:
# a set of antonym that are opposite along the same supposed dimension (es, gender male-female), and ..average of all the parallel lines

## Find the antonym difference vectors
antonymy_vectors = []
dim_descr = [] # also keep track of the antonym word pair on which each dimension is based
for each_word_pair in list_antonym:
    antonymy_vectors.append(current_model[each_word_pair[0]] - current_model[each_word_pair[1]])
    antonyms_pair = each_word_pair[0] + '-' + each_word_pair[1]
    dim_descr.append(antonyms_pair)
antonymy_vectors = np.array(antonymy_vectors)
print(antonymy_vectors.shape)
# todo: keep track of word pair names (antonym pair) and the corresponding diff vector; also add the ..score of orthogonality..
print(f'list_antonym[0]: {list_antonym[0]}, with antonymy difference vector head{(current_model[list_antonym[0][0]] - current_model[list_antonym[0][1]])[:5]}')
print(f'antonymy_vectors[0] head (5): {antonymy_vectors[0][:5]}')
print(f'dim_descr head (5): {dim_descr[:5]}')

(1468, 300)
list_antonym[0]: ('exclude', 'inscribe'), with antonymy difference vector head[ 0.12662849 -0.08837156 -0.14503697 -0.0768957   0.04391089]
antonymy_vectors[0] head (5): [ 0.12662849 -0.08837156 -0.14503697 -0.0768957   0.04391089]
dim_descr head (5): ['exclude-inscribe', 'assertion-meek', 'beginning-terminal', 'lop-secure', 'libertine-pastor']
time: 37.9 ms (started: 2022-02-19 20:02:03 +00:00)


## Subset Dimension Selection Method

In [45]:
import random

from scipy.spatial.distance import cosine as scipy_cosine
random.seed(42)

if os.path.exists(dimension_similarity_matrix_filename):
    dimension_similarity_matrix = npload(dimension_similarity_matrix_filename)
else:
  t1 = np.array(antonymy_vectors) # this are the difference vectors for all pair of antonyms
  dimension_similarity_matrix = defaultdict(dict)
  for index_1, each_dim1 in enumerate(tqdm(t1)):
      for index_2, each_dim2 in enumerate(t1):
          dimension_similarity_matrix[index_1][index_2] = abs(1-scipy_cosine(each_dim1, each_dim2))
  npsave(dimension_similarity_matrix, dimension_similarity_matrix_filename) 
  #todo: instead of index by numbers, use the word strings, eg index_1 is a word pair horizontal-vertical



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1468 [00:00<?, ?it/s]

time: 2min 12s (started: 2022-02-19 18:16:04 +00:00)


## Generate the ORTHOGONAL DIMENSION Order

In [84]:
def get_average_similarity(final_list, candidate_dimension):
    """
    Given a *final_list* of dimensions already choosen, and a candidate dimension *candidate_dimension*, 
    returns the mean of all the similarities btw candidate_dimension and all the final_list dimensions.
    """
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][candidate_dimension]
    return final_output/(len(final_list))

# todo, use max cosine instead, worst case similarity

def select_subset_dimension(dim_vector, num_dim):
    """
    Given a set of antonyms vectors (diff, directional vectors) returns ..
    """
    # todo, also keep track of the antonym word pair on which each dimension is based
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)

    # choose randomly one of the antonym dimensions, and set it as first
    # problem: shouldn't we use another ..metric to decide the most suitable 
    # "first dimension"? like in ..PCA is the one with most variance
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)
 
    for test_count in tqdm(range(num_dim-1)):
        most_distant_new_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
            # print(each_dim)
            temp_score = get_average_similarity(final_position_index, each_dim)
            if temp_score < min_score:
                min_score = temp_score
                most_distant_new_dim = each_dim
        # print(test_count,most_distant_new_dim)
        #print(f'appending choosing a dimension with average similarity to prev dims {min_score}')
        final_position_index.append(most_distant_new_dim)
        working_position_index.remove(most_distant_new_dim)
        # print(working_list.shape, len(final_list))
    return final_position_index


time: 34.4 ms (started: 2022-02-19 20:24:11 +00:00)


In [89]:

def print_similarities_among_dimensions_in_final_list(final_position_index):
    print(f'final subset of dimensions is:')
    for dim_idx in final_position_index[:5]:
      print(f'{dim_descr[dim_idx]}')

    min_angle_to_prev_dimensions = dict()
    min_angle_to_prev_dimensions[0] = 90
    orthogonality_infos = []
    
    for idx in tqdm(range(1, len(final_position_index))):
      dim_idx = final_position_index[idx]
      min_angle = 90
      closest_dim = None
      for idx2 in range(0, idx):
        dim2_idx = final_position_index[idx2]
        # nb dimension_similarity_matrix[v1][v2] = abs(1-scipy_cosine(v1, v2))
        distance = dimension_similarity_matrix[dim_idx][dim2_idx] 
        cosine_similarity = 1 - distance
        angle = from_cosine_to_degrees(cosine_similarity)
        if angle < min_angle:
          min_angle = angle
          closest_dim = dim2_idx
      min_angle_to_prev_dimensions[dim_idx] = min_angle
      orthogonality_info = {'idx': dim_idx, 'dim': dim_descr[dim_idx], 'idx2': dim2_idx, 'closest_dim': dim_descr[dim2_idx], 'angle_degrees': min_angle}
      orthogonality_infos.append(orthogonality_info)

    # ideally they should all be 90, which means all orthogonal dimensions
    print(f'min angles:')

    for orthogonality_info in list(reversed(sorted(orthogonality_infos, key=lambda x: x['angle_degrees'])))[:5]:
      print(f'{orthogonality_info}')

num_antonym = 1443
if os.path.exists(orthogonal_antonymy_vector_filename):
  orthogonal_antonymy_vector = npload(orthogonal_antonymy_vector_filename)
  subset_dimensions = orthogonal_antonymy_vector.tolist()
else:
  subset_dimensions = select_subset_dimension(antonymy_vectors, num_antonym)
  orthogonal_antonymy_vector = np.array(subset_dimensions)  
  npsave(orthogonal_antonymy_vector, orthogonal_antonymy_vector_filename)

print(orthogonal_antonymy_vector.shape)
print(orthogonal_antonymy_vector)
print(len(antonymy_vectors[0]))
print(f'list_antonym len {len(list_antonym)} {list_antonym}')

print_similarities_among_dimensions_in_final_list(subset_dimensions)

import collections
ortogonal_antonyms_by_score = dict()
for antonym_idx, score in enumerate(orthogonal_antonymy_vector):
  ortogonal_antonyms_by_score[score] = list_antonym[antonym_idx]
sorted_ortogonal_antonyms = collections.OrderedDict(sorted(ortogonal_antonyms_by_score.items()))
print(f'sorted_ortogonal_antonyms {list(sorted_ortogonal_antonyms.items())[:15]}')
print(f'sorted_ortogonal_antonyms last elements: {list(reversed(sorted_ortogonal_antonyms.items()))[:15]}')


(1443,)
[ 563  485 1275 ...  669  136  911]
300
list_antonym len 1468 [('exclude', 'inscribe'), ('assertion', 'meek'), ('beginning', 'terminal'), ('lop', 'secure'), ('libertine', 'pastor'), ('hook', 'pocket'), ('orbital', 'stable'), ('disprove', 'document'), ('biological', 'psychological'), ('preparedness', 'unreadiness'), ('dehydration', 'saturation'), ('biologic', 'geologic'), ('accessibility', 'hindrance'), ('agree', 'crab'), ('beautify', 'misconstrue'), ('lump', 'separate'), ('double', 'indivisible'), ('forgetting', 'training'), ('determined', 'irresolute'), ('still', 'velocity'), ('shield', 'unshield'), ('guess', 'tabulate'), ('iconic', 'undefined'), ('caress', 'punch'), ('compressor', 'stretcher'), ('allude', 'specify'), ('birthplace', 'cemetery'), ('corrupt', 'sanctify'), ('actor', 'cameraman'), ('knowledgeable', 'naivety'), ('screech', 'talk'), ('hedge', 'squander'), ('aggregate', 'scatter'), ('constrict', 'free'), ('break', 'construct'), ('descriptive', 'normative'), ('inscrib

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1442 [00:00<?, ?it/s]

min angles:
{'idx': 1255, 'dim': 'complex-plain', 'idx2': 773, 'closest_dim': 'economics-psychology', 'angle_degrees': 6.056120438293147}
{'idx': 1053, 'dim': 'long-second', 'idx2': 688, 'closest_dim': 'air-earth', 'angle_degrees': 5.601147260906531}
{'idx': 731, 'dim': 'cynicism-naivety', 'idx2': 655, 'closest_dim': 'pat-swat', 'angle_degrees': 5.5220822723154654}
{'idx': 583, 'dim': 'tomato-township', 'idx2': 1275, 'closest_dim': 'come-leave', 'angle_degrees': 5.272363537470117}
{'idx': 1399, 'dim': 'shadow-sunlight', 'idx2': 602, 'closest_dim': 'stationary-velocity', 'angle_degrees': 5.18551029946702}
sorted_ortogonal_antonyms [(0, ('enjoyment', 'pain')), (1, ('caress', 'punch')), (2, ('lace', 'silk')), (3, ('electron', 'proton')), (4, ('accountable', 'despotic')), (5, ('complicate', 'plain')), (6, ('expose', 'hedge')), (7, ('sheep', 'wolf')), (8, ('certain', 'uninsured')), (9, ('game', 'reality')), (10, ('encrypt', 'interpret')), (11, ('construe', 'keep')), (12, ('freestyle', 'stru

In [ ]:
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity

dotproduct = 29
denom = math.sqrt(13) * math.sqrt(74)
cosine = dotproduct / denom
angle = math.degrees(np.arccos(cosine))
print(f'cosine {cosine}, angle {angle}')

def custom_cosine_sim(vec1, vec2):
  dotproduct = np.dot(vec1, vec2)
  denom = np.linalg.norm(vec1) * np.linalg.norm(vec2)
  cosine = dotproduct / denom
  angle = np.arccos(cosine)
  #print(f'cosine {cosine}, angle {angle}')
  return cosine

def print_cos_and_angle(angle):
  angle_radians = math.radians(angle)
  cosine = math.cos(angle_radians)
  print(f'cos {angle} {cosine:.3f}, arccos {from_cosine_to_degrees(cosine):.3f}')  

def from_cosine_to_degrees(cosine):
  return math.degrees(np.arccos(cosine))

# todo two sample vectors [2,3] [7,5], check cosine distance and angle
# compare with results from library cosine distance


p1 = np.array([2,3])
p2 = np.array([7,5])
print(f'numpy dot: {np.dot(p1, p2)}')
print(f'custom_cosine_sim: {custom_cosine_sim(p1,p2)}')

cosine_sim = cosine_similarity([p1],[p2])
print(f'cosine similarity btw {p1} and {p2}: {cosine_sim}')
print(f'cosine similarity {cosine_sim} corresponds to degrees {from_cosine_to_degrees(cosine_sim)}')
abs_cosine_sim = abs(cosine_sim[0][0])
print_cos_and_angle(0)
print_cos_and_angle(45)
print_cos_and_angle(60)
print_cos_and_angle(90)
print_cos_and_angle(180)
print_cos_and_angle(270)

## Generate the RANDOM DIMENSION Order

In [ ]:

random_antonym_vector = [i for i in range(len(antonymy_vectors))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

## Generate the MAXIMUM VARIANCE DIMENSION Order

In [ ]:

!ls -lh 
!ls -lh ./embedding_matrix_np/
gc.collect()
#os.remove('embedding_matrix_np')

embedding_size = antonymy_vectors.shape[0]   
print('The embedding size is', embedding_size)
vocab_size = len(current_model.vocab)
print(f'vocab_size {vocab_size}')

def human_readable_size(size, decimal_places=1):
    for unit in ['B', 'KiB', 'MiB', 'GiB', 'TiB', 'PiB']:
        if size < 1024.0 or unit == 'PiB':
            break
        size /= 1024.0
    return f"{size:.{decimal_places}f} {unit}"


def save_embedding_matrix_method1(embedding_matrix_buffer, embedding_matrix_np):
    batch_size = 10000

    if (not total_words % batch_size) or (total_words == vocab_size):
      print(f'saving embedding_matrix_buffer after {total_words} words, embedding_matrix_buffer mem size is {human_readable_size(sys.getsizeof(embedding_matrix_buffer))}')
      if not total_words % (batch_size*5):
        # todo: only call gc (on all gen) when memory use is above 10 gb
        gc.collect(generation=0)
      # convert to np arra
      embedding_matrix_buffer_np = np.array(embedding_matrix_buffer)
      if embedding_matrix_np is None:
        embedding_matrix_np = embedding_matrix_buffer_np
      else:
        # load file if existing, vstack buffered,
        embedding_matrix_np = npload(embedding_matrix_np_filename)
        embedding_matrix_np = np.vstack((embedding_matrix_np,embedding_matrix_buffer_np))
      #  and resave
      print(f'saving embedding_matrix_np with shape: {embedding_matrix_np.shape}, and mem size {human_readable_size(sys.getsizeof(embedding_matrix_np))}')
      npsave(embedding_matrix_np, embedding_matrix_np_filename)
      embedding_matrix_buffer = []

def save_embedding_matrix_method2():
  variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vectors))


  embedding_matrix_buffer = []
  embedding_matrix_np = None
  embedding_matrix_files = []

  # todo, use batch size 100k, save each 100k np array to a separate file
  # reconstruct the file later at the end of the loop, save to a single file
  # check it can fit into memory

  total_words = 0
  for each_word in tqdm(current_model.vocab):
      total_words += 1

      new_vector = np.matmul(variance_antonymy_vector_inverse,current_model[each_word])
      
      embedding_matrix_buffer.append(new_vector)
      # save_embedding_matrix_method2(embedding_matrix_buffer, embedding_matrix_np, embedding_matrix_files)
      batch_size = 200000

      if (not total_words % batch_size) or (total_words == vocab_size):

        suffix = len(embedding_matrix_files)
        current_embedding_matrix_np_filename = embedding_matrix_np_filename + '_' + str(suffix) + '.pkl'
        current_embedding_matrix_np_path = os.path.join(embedding_matrix_np_dir, current_embedding_matrix_np_filename)
        if not os.path.exists(embedding_matrix_np_dir):
          os.mkdir(embedding_matrix_np_dir)
        print(f'saving to file {current_embedding_matrix_np_path}')

        print(f'saving embedding_matrix_buffer after {total_words} words, embedding_matrix_buffer mem size is {human_readable_size(sys.getsizeof(embedding_matrix_buffer))}')
        embedding_matrix_buffer_np = np.array(embedding_matrix_buffer)

        print(f'saving embedding_matrix_np with shape: {embedding_matrix_buffer_np.shape}, and mem size {human_readable_size(sys.getsizeof(embedding_matrix_np))}')
        npsave(embedding_matrix_buffer_np, current_embedding_matrix_np_path)
        embedding_matrix_files.append(current_embedding_matrix_np_path)
        embedding_matrix_buffer = []
        gc.collect()

# todo: check if dir contains already 10 files
if not os.path.exists(embedding_matrix_np_dir):
  embedding_matrix_already_generated = False
else:
  embedding_matrix_already_generated = len(os.listdir(embedding_matrix_np_dir)) >= 10
if not embedding_matrix_already_generated:
  save_embedding_matrix_method2()

In [ ]:
!ls -lh ./embedding_matrix_np/
print(f'len(os.listdir(embedding_matrix_np_dir)) {len(os.listdir(embedding_matrix_np_dir))}')

In [ ]:
del new_vector

In [ ]:
!ls -lh ./embedding_matrix_np/
variance_list = []

embedding_matrix = npload(embedding_matrix_np_filename)
# embedding_matrix = np.array(embedding_matrix)

for each_dimension in tqdm(range(embedding_matrix.shape[1])):
    variance_list.append(np.var(embedding_matrix[:,each_dimension]))

In [ ]:
variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]

In [ ]:
del embedding_matrix

In [ ]:
del variance_list

# Transformation to polar space

In [ ]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):
    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model.vocab), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model.vocab))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model.vocab))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model.vocab):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)

        
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()



# Standard normal transform 

In [ ]:
def standard_normal_dist_model(model, new_filename):
    embedding_matrix = []
    embedding_vocab = []

    temp_file = open(new_filename,'wb')
    temp_file.write(str.encode(str(model.vectors.shape[0])+' '+str(model.vectors.shape[1])+'\n'))
    
    for each_word in tqdm(model.vocab):
        embedding_matrix.append(model[each_word])
        embedding_vocab.append(each_word)
    
    embedding_matrix = np.array(embedding_matrix)
    
    print('The shape of embedding matrix is {}'.format(embedding_matrix.shape))
    
    norm_embedding_matrix = (embedding_matrix - embedding_matrix.mean(0))/ embedding_matrix.std(0)
    
    for word_counter, each_word in enumerate(tqdm(embedding_vocab)):
#         assert each_word==embedding_vocab[word_counter],'Not matching!!!'
        
        temp_file.write(str.encode(each_word+' '))
        new_vector = norm_embedding_matrix[word_counter]
        temp_file.write(new_vector)
        temp_file.write(str.encode('\n'))
        
    del embedding_matrix
    del embedding_vocab
    temp_file.close()

## Compute the task score for different dimension size

In [ ]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vectors[antonym_vectors[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

In [ ]:
import subprocess
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = random_antonym_vector # orthogonal_antonymy_vector, variance_antonymy_vector

In [ ]:
curr_dim = dim_size

embedding_path = 'Downstream Task/embeddings/rand_antonym_'+str(curr_dim)+'.bin'
generate_embedding_path(current_model, embedding_path,True,random_antonym_vector, curr_dim)

print('loading the model')
temp_model = gensim.models.KeyedVectors.load_word2vec_format(embedding_path, binary=True)
print('loading done..')

std_nrml_embedding_path = 'Downstream Task/embeddings/rand_antonym_gl_'+str(curr_dim)+'_StdNrml.bin'
standard_normal_dist_model(temp_model, std_nrml_embedding_path)

######### Task specific code....
command_list = ['python3', 
                    'Downstream Task/TREC/classify_task.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/TREC/data/qa_train_X.pickle',
                     'Downstream Task/TREC/data/qa_train_y.pickle',
                     'Downstream Task/TREC/data/qa_val_X.pickle',
                     'Downstream Task/TREC/data/qa_val_y.pickle',
                     'Downstream Task/TREC/data/qa_test_X.pickle',
                     'Downstream Task/TREC/data/qa_test_y.pickle'
                    ]
result = subprocess.run(command_list, stdout=subprocess.PIPE)
print(result.stdout)
print('finished sentiment task')
################
del temp_model
    
os.remove(std_nrml_embedding_path)
os.remove(embedding_path)

# Code snippet to use for other downstream tasks

In [ ]:

## word analogy....
######################################################################   
    
    del temp_model
    
    temp_model_nrml = gensim.models.KeyedVectors.load_word2vec_format(std_nrml_embedding_path, binary=True)
    accuracy = temp_model_nrml.evaluate_word_analogies(datapath('questions-words.txt'))[0]
    
    print(accuracy)
    final_output_wa.write(str(curr_dim)+'\tstdnrml\t')
    final_output_wa.write(str(accuracy))
    final_output_wa.write('\n')
    final_output_wa.flush()
    
    del temp_model_nrml
######################################################################
### Sentiment classification    
    command_list = ['python3', 
                    'Downstream Task/sentiment/classify_sentiment.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/sentiment/data/sentiment_train_X.p',
                     'Downstream Task/sentiment/data/sentiment_train_y.p',
                     'Downstream Task/sentiment/data/sentiment_val_X.p',
                     'Downstream Task/sentiment/data/sentiment_val_y.p',
                     'Downstream Task/sentiment/data/sentiment_test_X.p',
                     'Downstream Task/sentiment/data/sentiment_test_y.p'
                    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished sentiment task')




######################################################################
### Question classification
    command_list = ['python3', 
                    'Downstream Task/TREC/classify_task.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/TREC/data/qa_train_X.pickle',
                     'Downstream Task/TREC/data/qa_train_y.pickle',
                     'Downstream Task/TREC/data/qa_val_X.pickle',
                     'Downstream Task/TREC/data/qa_val_y.pickle',
                     'Downstream Task/TREC/data/qa_test_X.pickle',
                     'Downstream Task/TREC/data/qa_test_y.pickle'
                    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished TREC task')
    

#####################################################################
### Discriminative attribute identification
    command_list = ['python3', 
                    'Downstream Task/Discrim_Attr/classify_discrim_attr_TASK.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_train_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_train_y.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_val_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_val_y.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_test_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_test_y.p'
                   ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Discr task')
    
######################################################################
### Computer news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_computer_train_X.p',
                    'Downstream Task/newsgroups/data/news_computer_train_y.p',
                    'Downstream Task/newsgroups/data/news_computer_val_X.p',
                    'Downstream Task/newsgroups/data/news_computer_val_y.p',
                    'Downstream Task/newsgroups/data/news_computer_test_X.p',
                    'Downstream Task/newsgroups/data/news_computer_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished compter nc task')

######################################################################   
### Religious news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_religion_train_X.p',
                    'Downstream Task/newsgroups/data/news_religion_train_y.p',
                    'Downstream Task/newsgroups/data/news_religion_val_X.p',
                    'Downstream Task/newsgroups/data/news_religion_val_y.p',
                    'Downstream Task/newsgroups/data/news_religion_test_X.p',
                    'Downstream Task/newsgroups/data/news_religion_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Religion nc task')

######################################################################
### sports news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_sports_train_X.p',
                    'Downstream Task/newsgroups/data/news_sports_train_y.p',
                    'Downstream Task/newsgroups/data/news_sports_val_X.p',
                    'Downstream Task/newsgroups/data/news_sports_val_y.p',
                    'Downstream Task/newsgroups/data/news_sports_test_X.p',
                    'Downstream Task/newsgroups/data/news_sports_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Sports nc task')

    
######################################################################  
### noun phrase bracketing
    res = []
    for _ in range(10):
        command_list = ['python3',
                        'Downstream Task/np_bracketing/classify_bracketing.py',
                        std_nrml_embedding_path,
                        '2',
                        'Downstream Task/np_bracketing/data/npbracketing_train_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_train_y'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_val_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_val_y'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_test_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_test_y'+str(_)+'.pickle'
        ]

        result = subprocess.run(command_list, stdout=subprocess.PIPE)
        print(result.stdout)
        res.append(float(result.stdout.decode('utf-8').strip()))
    
    #print(res)
    m_s = np.mean(res)
    print(m_s)
